# Conversation Management & JSON Schema Extraction (Groq OpenAI-Compatible)
# Author: Md. Hasan Imon
# Objective:
# - Task 1: Conversation management + summarization
# - Task 2: JSON schema classification & extraction
# Notes:
# - Framework-free: only Python + requests + openai client + jsonschema
# - Uses Groq API (OpenAI-compatible endpoint)


In [16]:
!pip install --quiet openai jsonschema


## Securely provide your Groq API key

In [ ]:
from getpass import getpass
import os

# Hidden input in Colab
GROQ_API_KEY = getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


## Initialize OpenAI-compatible client pointing at Groq


In [53]:
import openai

client = openai.OpenAI(
    api_key=os.environ["GROQ_API_KEY"],
    base_url="https://api.groq.com/openai/v1"
)

MODEL_NAME = "openai/gpt-oss-20b"
print("Groq client initialized with model:", MODEL_NAME)

Groq client initialized with model: openai/gpt-oss-20b



## Conversation Manager Class


In [54]:
import time
import json

class ConversationManager:
    def __init__(self, model_name: str, client, summary_every_k: int = 3):
        self.model_name = model_name
        self.client = client
        self.history = []
        self.run_counter = 0
        self.summary_every_k = summary_every_k
        self.summaries = []

    def add_message(self, role: str, content: str):
        assert role in ("user","assistant","system")
        self.history.append({"role": role, "content": content})

    def get_last_n_turns(self, n:int):
        return self.history[-2*n:] if 2*n <= len(self.history) else self.history[:]

    def truncate_by_chars(self, max_chars:int):
        kept = []
        total = 0
        for msg in reversed(self.history):
            l = len(msg["content"])
            if total + l > max_chars:
                break
            kept.append(msg)
            total += l
        self.history = list(reversed(kept))

    def truncate_by_words(self, max_words:int):
        kept = []
        total = 0
        for msg in reversed(self.history):
            w = len(msg["content"].split())
            if total + w > max_words:
                break
            kept.append(msg)
            total += w
        self.history = list(reversed(kept))

    def summarize_history(self, summary_prompt_extra=""):
        conversation_text = "\n".join([f"{m['role'].upper()}: {m['content']}" for m in self.history])
        system_msg = {
            "role": "system",
            "content": "You are a helpful summarizer. Produce a concise summary of the conversation."
        }
        user_msg = {
            "role": "user",
            "content": f"Summarize the following conversation into a short, factual summary (6-12 bullet points). {summary_prompt_extra}\n\nConversation:\n{conversation_text}"
        }
        resp = self.client.chat.completions.create(
            model=self.model_name,
            messages=[system_msg, user_msg],
            max_tokens=512,
            temperature=0.0
        )
        summary_text = ""
        if hasattr(resp, "choices") and len(resp.choices) > 0:
            summary_text = resp.choices[0].message.content # Corrected access
        ts = int(time.time())
        self.summaries.append({"timestamp": ts, "summary": summary_text})
        self.history = [{"role":"system", "content": f"[AUTO-SUMMARY at {time.ctime(ts)}]\n{summary_text}"}]
        return summary_text

    def process_user_message(self, user_text: str, do_summarize_if_needed=True):
        self.add_message("user", user_text)
        resp = self.client.chat.completions.create(
            model=self.model_name,
            messages=self.history + [{"role":"user","content":user_text}],
            max_tokens=512,
            temperature=0.0
        )
        assistant_text = ""
        if hasattr(resp, "choices") and len(resp.choices) > 0:
            assistant_text = resp.choices[0].message.content # Corrected access
        self.add_message("assistant", assistant_text)
        self.run_counter += 1
        summary = None
        if do_summarize_if_needed and self.summary_every_k > 0 and self.run_counter % self.summary_every_k == 0:
            summary = self.summarize_history()
        return assistant_text, summary

## 5) Demonstration of Task 1: multiple conversation samples


In [55]:
cm = ConversationManager(model_name=MODEL_NAME, client=client, summary_every_k=3)

samples = [
    "Hi, I want to prepare a project-based portfolio for AI agent jobs. Can you help?",
    "I have experience with LangGraph and LangChain. I need advanced project ideas.",
    "Suggest a multi-agent architecture for a RAG-based customer support assistant.",
    "How to implement fallback logic and tool routing?",
    "Which vector DB, and how to store short-term vs long-term memory effectively?",
    "Show me a concise plan for deployment and CI/CD for such a multi-agent app."
]

for i, s in enumerate(samples, start=1):
    assistant_text, summary = cm.process_user_message(s)
    print(f"\n=== Turn {i} ===")
    print("User:", s)
    print("Assistant (truncated):", (assistant_text[:400] + "...") if len(assistant_text)>400 else assistant_text)
    if summary:
        print("\n--- AUTO SUMMARY GENERATED ---")
        print(summary[:800])
        print("-----------------------------")

print("\n=== Current History Stored ===")
for m in cm.history:
    print(f"{m['role'].upper()}: {m['content'][:400]}...\n")


=== Turn 1 ===
User: Hi, I want to prepare a project-based portfolio for AI agent jobs. Can you help?
Assistant (truncated): 

=== Turn 2 ===
User: I have experience with LangGraph and LangChain. I need advanced project ideas.
Assistant (truncated): 

=== Turn 3 ===
User: Suggest a multi-agent architecture for a RAG-based customer support assistant.
Assistant (truncated): ## 🚀 Multi‑Agent RAG‑Based Customer Support Assistant  
*(Designed for LangChain + LangGraph, but agnostic to the underlying LLM provider)*  

Below is a **complete, production‑ready architecture** that splits the customer‑support workflow into a set of cooperating agents. Each agent has a single, well‑defined responsibility, communicates via a lightweight message bus, and can be swapped, scaled, ...

--- AUTO SUMMARY GENERATED ---
- User seeks help building a project‑based portfolio for AI agent roles.  
- User has experience with LangGraph and LangChain and wants advanced project ideas.  
- User specifically reques

### Demonstrate truncation options

In [56]:
# Re-populate some history
cm.history = []
for i in range(1,9):
    cm.add_message("user", f"User question #{i} — details about task {i}.")
    cm.add_message("assistant", f"Assistant reply #{i} — answers and clarifications.")

print("\n-- Last 2 turns --")
for m in cm.get_last_n_turns(2):
    print(m)

print("\n-- Truncate by chars = 200 --")
cm.truncate_by_chars(200)
for m in cm.history:
    print(m)

cm.history = []
for i in range(1,9):
    cm.add_message("user", f"User question #{i} — details about task {i}.")
    cm.add_message("assistant", f"Assistant reply #{i} — answers and clarifications.")

print("\n-- Truncate by words = 40 --")
cm.truncate_by_words(40)
for m in cm.history:
    print(m)


-- Last 2 turns --
{'role': 'user', 'content': 'User question #7 — details about task 7.'}
{'role': 'assistant', 'content': 'Assistant reply #7 — answers and clarifications.'}
{'role': 'user', 'content': 'User question #8 — details about task 8.'}
{'role': 'assistant', 'content': 'Assistant reply #8 — answers and clarifications.'}

-- Truncate by chars = 200 --
{'role': 'user', 'content': 'User question #7 — details about task 7.'}
{'role': 'assistant', 'content': 'Assistant reply #7 — answers and clarifications.'}
{'role': 'user', 'content': 'User question #8 — details about task 8.'}
{'role': 'assistant', 'content': 'Assistant reply #8 — answers and clarifications.'}

-- Truncate by words = 40 --
{'role': 'assistant', 'content': 'Assistant reply #6 — answers and clarifications.'}
{'role': 'user', 'content': 'User question #7 — details about task 7.'}
{'role': 'assistant', 'content': 'Assistant reply #7 — answers and clarifications.'}
{'role': 'user', 'content': 'User question #8 — d

## Task 2: JSON Schema Classification & Extraction


In [57]:
from jsonschema import validate, ValidationError

schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "email": {"type": "string", "format": "email"},
        "phone": {"type": "string"},
        "location": {"type": "string"},
        "age": {"type": "integer", "minimum": 0, "maximum": 150}
    },
    "required": ["name"],
    "additionalProperties": False
}

functions = [
    {
        "name": "extract_user_info",
        "description": "Extract contact and basic personal info from the conversation or message.",
        "parameters": schema
    }
]


### Helper: function-calling model


In [58]:
import re
def call_function_calling_model(client, model_name, user_message, functions):
    messages = [
        {"role":"system","content":"You are a JSON extractor. Call the provided function with structured JSON when possible."},
        {"role":"user","content":user_message}
    ]
    resp = client.chat.completions.create(
        model=model_name,
        messages=messages,
        functions=functions,
        function_call="auto",
        max_tokens=512,
        temperature=0.0
    )
    if hasattr(resp, "choices") and len(resp.choices) > 0:
        msg = resp.choices[0].message
        if hasattr(msg, "function_call") and msg.function_call:
            fname = msg.function_call.name # Corrected access
            fargs_raw = msg.function_call.arguments # Corrected access
            try:
                fargs = json.loads(fargs_raw)
            except Exception:
                json_text_match = re.search(r"(\{.*\})", fargs_raw, re.S)
                if json_text_match:
                    fargs = json.loads(json_text_match.group(1))
                else:
                    fargs = None
            return fname, fargs, resp
    if hasattr(resp, "choices") and len(resp.choices) > 0:
        txt = resp.choices[0].message.content # Corrected access
        json_text_match = re.search(r"(\{.*\})", txt, re.S)
        if json_text_match:
            try:
                parsed = json.loads(json_text_match.group(1))
                return None, parsed, resp
            except:
                pass
    return None, None, resp


### Sample chats for extraction


In [59]:
sample_chats = [
    "Hello, I'm Md. Hasan Imon. You can reach me at emon.mlengineer@gmail.com. My phone is +8801834363533. I live in Savar, Dhaka. I'm 24 years old.",
    "Hey there — name's Emon. I'm 25 and currently living near Dhaka city. Email: md.emon.hasan@example.com. Call me maybe 01834363533",
    "Hi, this is Ayesha from Chittagong. I'm in my early 30s. My email is ayesha.work@mailprovider.com. Don't have a phone right now."
]

results = []
for i, chat in enumerate(sample_chats, start=1):
    fname, fargs, raw = call_function_calling_model(client, MODEL_NAME, chat, functions)
    print(f"\n--- Sample {i} ---")
    print("Input:", chat)
    print("Function called:", fname)
    print("Parsed args:", fargs)
    valid = False
    errors = None
    if fargs:
        try:
            validate(instance=fargs, schema=schema)
            valid = True
        except ValidationError as e:
            errors = str(e)
    print("Validation:", "OK" if valid else f"FAILED: {errors}")
    results.append({"input":chat, "function":fname, "parsed":fargs, "valid":valid, "errors":errors})



--- Sample 1 ---
Input: Hello, I'm Md. Hasan Imon. You can reach me at emon.mlengineer@gmail.com. My phone is +8801834363533. I live in Savar, Dhaka. I'm 24 years old.
Function called: extract_user_info
Parsed args: {'age': 24, 'email': 'emon.mlengineer@gmail.com', 'location': 'Savar, Dhaka', 'name': 'Md. Hasan Imon', 'phone': '+8801834363533'}
Validation: OK

--- Sample 2 ---
Input: Hey there — name's Emon. I'm 25 and currently living near Dhaka city. Email: md.emon.hasan@example.com. Call me maybe 01834363533
Function called: extract_user_info
Parsed args: {'age': 25, 'email': 'md.emon.hasan@example.com', 'location': 'Dhaka city', 'name': 'Emon', 'phone': '01834363533'}
Validation: OK

--- Sample 3 ---
Input: Hi, this is Ayesha from Chittagong. I'm in my early 30s. My email is ayesha.work@mailprovider.com. Don't have a phone right now.
Function called: extract_user_info
Parsed args: {'age': 30, 'email': 'ayesha.work@mailprovider.com', 'location': 'Chittagong', 'name': 'Ayesha'}
Vali